In [1]:
import os
import sys

from glob import glob
from typing import List, Tuple

import numpy as np
import xarray as xr
import xcdat as xc
import xskillscore as xscore

# Personal Data Loader
sys.path.append('../pipeline')
from data_loader import DataLoader
from data_transformer import DataTransformer
from run_dataloader import era5_single_level, era5_pressure_level

/glade/work/zespinosa/conda-envs/cenv/lib/python3.8/site-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.31.0 or higher is recommended. You are running version 2.27.0
  warnings.warn(


In [12]:
REF_PERIOD = ("1980-01-01", "2020-01-01")
TESTING = True
SAVE = True

In [3]:
dataloader = DataLoader(
    root = [
        "/glade/campaign/univ/uwas0118/scratch/archive/1950_2015/",
        "/glade/scratch/zespinosa/archive/cesm2.1.3_BHISTcmip6_f09_g17_ERA5_nudge/", 
        "/glade/scratch/zespinosa/archive/cesm2.1.3_BSSP370cmip6_f09_g17_ERA5_nudge/"
    ],
    era5_root="/glade/work/zespinosa/data/era5/monthly"
)

In [ ]:
era5_single = era5_single_level(dataloader)
era5_pressure = era5_pressure_level(dataloader)

In [4]:
datatransformer = DataTransformer(
    save_path='/glade/work/zespinosa/Projects/SI-Antarctic/data'
)

In [5]:
def transform_era5_data(era5_data, test, save, cvar):

    if test: 
        era5_data = era5_data.sel(time=slice("1979-01-01", "1980-01-01"))

    # Regrid and save
    print("starting regrid")
    era5_data = datatransformer.regrid(ds=era5_data, save=save, save_name=f"ERA5_monthly_1979-01_2023-12_{cvar}")

    # Calculate anomalies and climatology
    print("starting anomalies and climatology")
    era5_data_ac = datatransformer.calculate_anoms_climatology(
        ds=era5_data.copy(),
        ref_period=REF_PERIOD,
        save_name=f"ERA5_monthly_1979-01_2023-12_{cvar}",
        save=save,
    )

    print("starting trends")
    era5_data_trends = datatransformer.calculate_linear_time_trend(
        ds=era5_data.copy(),
        save=True,
        save_name=f"ERA5_monthly_1979-01_2023-12_{cvar}",
    )

transform_era5_data(era5_single, test=False, save=True, cvar="full-single")
transform_era5_data(era5_pressure, test=False, save=True, cvar="full-pressure")


starting regrid


starting anomalies and climatology
starting trends
> /glade/scratch/zespinosa/ipykernel_14321/4022574871.py(21)transform_era5_data()
     19     print("starting trends")
     20     import pdb; pdb.set_trace()
---> 21     era5_data_trends = datatransformer.calculate_linear_time_trend(
     22         ds=era5_data.copy(),
     23         save=True,

starting regrid
starting anomalies and climatology
starting trends
> /glade/scratch/zespinosa/ipykernel_14321/4022574871.py(21)transform_era5_data()
     19     print("starting trends")
     20     import pdb; pdb.set_trace()
---> 21     era5_data_trends = datatransformer.calculate_linear_time_trend(
     22         ds=era5_data.copy(),
     23         save=True,



In [15]:
def transform_cesm2_data(ds, save, save_name, testing):
    print("starting regrid")
    ds = datatransformer.regrid(
        ds, 
        save=save, 
        save_name=save_name
    )

    print("starting anomalies and climatology")
    ds_ac = datatransformer.calculate_anoms_climatology(
        ds, 
        ref_period=REF_PERIOD, 
        save=save, 
        save_name=save_name
    )

    print("starting trends")
    ds_trend = datatransformer.calculate_linear_time_trend(
        ds, 
        save=save,
        save_name=save_name
    )


In [16]:
ice_cesm2 = dataloader.get_cesm2_data(
    comp="ice",
    myvars=["aice", "daidtt", "daidtd", "dvidtt", "dvidtd", "sithick", "uvel", "vvel"],
    testing=TESTING,
)

transform_cesm2_data(
    ds=ice_cesm2,
    save=SAVE,
    testing=TESTING,
    save_name="cesm2_ice_monthly_1950-01_2023-12",
)

Loading ice data from 900 files...
starting regrid


/glade/work/zespinosa/conda-envs/cenv/lib/python3.8/site-packages/xarray/conventions.py:450: UserWarning: Variable 'time' has datetime type and a bounds variable but time.encoding does not have units specified. The units encodings for 'time' and 'time_bnds' will be determined independently and may not be equal, counter to CF-conventions. If this is a concern, specify a units encoding for 'time' before writing to a file.
  warnings.warn(


starting anomalies and climatology


ValueError: month must not be empty

In [ ]:
ocn_mxl = dataloader.get_data(comp="ocn", myvars=["HMXL"], testing=testing)
transform_cesm2_data(
    ds=ocn_mxl,
    save=SAVE,
    testing=TESTING,
    save_name="cesm2_ocn-mxl_monthly_1950-01_2023-12",
)

In [ ]:
ocn_sst = dataloader.get_data(comp="ocn", myvars=["SST"], testing=testing)
transform_cesm2_data(
    ds=ocn_sst,
    save=SAVE,
    testing=TESTING,
    save_name="cesm2_ocn-sst_monthly_1950-01_2023-12",
)

In [ ]:
atm_cesm2 = dataloader.get_cesm2_data(
    comp="atm", 
    myvars=["PSL", "U10", "TS", "T", "U", "V", "Z3"], 
    levels=[1000, 850, 500],
    testing=TESTING
)
transform_cesm2_data(
    ds=atm_cesm2,
    save=SAVE,
    testing=TESTING,
    save_name="cesm2_atm_monthly_1950-01_2023-12",
)


In [ ]:
def transform_cesm2_ice(dataloader, save, save_name, testing):
    ice_cesm2 = dataloader.get_cesm2_data(
        comp="ice",
        myvars=["aice", "daidtt", "daidtd", "dvidtt", "dvidtd", "sithick", "uvel", "vvel"],
        testing=testing
    )

    print("starting regrid")
    ice_cesm2 = dataloader.regrid(
        ice_cesm2, 
        save=save, 
        save_name=save_name
    )

    print("starting anomalies and climatology")
    ice_cesm2_ac = dataloader.calculate_anoms_climatology(
        ice_cesm2, 
        ref_period=REF_PERIOD, 
        save=save, 
        save_name=save_name
    )

    print("starting trends")
    ice_cesm2_trend = dataloader.calculate_linear_time_trend(
        ice_cesm2, 
        save=save,
        save_name=save_name
    )


In [ ]:
def transform_cesm2_ocn_mxl(dataloader, save, save_name, testing):
    ocn_mxl = dataloader.get_data(comp="ocn", myvars=["HMXL"], testing=testing)

    print("starting regrid")
    ocn_mxl = dataloader.regrid(
        ocn_mxl,  
        save=save, 
        save_name=save_name
    )

    print("starting anomalies and climatology") 
    ocn_mxl_ac = dataloader.calculate_anoms_climatology(
        ocn_mxl, 
        ref_period=REF_PERIOD, 
        save=save, 
        save_name=save_name
    )
    print("starting trends")
    ocn_mxl_trends = dataloader.calculate_linear_time_trend(
        ocn_mxl,
        save=save, 
        save_name=save_name
    )


In [ ]:
def transform_cesm2_ocn_sst(dataloader, save, save_name, testing):
    ocn_sst = dataloader.get_data(comp="ocn", myvars=["SST"], testing=testing)

    print("starting regrid")
    ocn_sst = dataloader.regrid(
        ocn_sst,  
        save=save, 
        save_name=save_name
        )

    print("starting anomalies and climatology") 
    ocn_sst_ac = dataloader.calculate_anoms_climatology(
        ocn_sst, 
        ref_period=REF_PERIOD, 
        save=save, 
        save_name=save_name
    )
    
    print("starting trends")
    ocn_sst_trends = dataloader.calculate_linear_time_trend(
        ocn_sst,
        save=save, 
        save_name=save_name
    )

In [ ]:
def transform_cesm2_atm(dataloader, save, save_name, testing):
    atm_cesm2 = dataloader.get_cesm2_data(
        comp="atm", 
        myvars=["PSL", "U10", "TS", "T", "U", "V", "Z3"], 
        levels=[1000, 850, 500],
        testing=testing
    )

    print("starting regrid")
    atm_cesm2 = dataloader.regrid(
        atm_cesm2, 
        save=save, 
        save_name=save_name
    )

    print("starting anomalies and climatology")
    atm_cesm2_ac = dataloader.calculate_anoms_climatology(
        atm_cesm2, 
        ref_period=REF_PERIOD, 
        save=save, 
        save_name=save_name
    )

    print("starting trends")
    atm_cesm2_trends = dataloader.calculate_linear_time_trend(
        atm_cesm2,
        save=save, 
        save_name=save_name
    )